In [ ]:
# Import necessary libraries
import os
from matplotlib import pyplot as plt
import cv2 as cv
import yaml

In [ ]:
# Visualize some images from the dataset
fig, ax = plt.subplots(1, 4, figsize=(15, 5))

# Paths to some example images
example_images = [
    "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/train/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-100_jpg.rf.3e853e83a0ba6d0109b4b60b0eb4a43a.jpg",
    "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/train/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-100_jpg.rf.ae2afb52f4b92561e39b05a9a733d0a5.jpg",
    "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/train/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-101_jpg.rf.22a1b9c7e51aa50ab420cdc8a9f0f576.jpg",
    "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/train/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-101_jpg.rf.f25d28218ff8e1616526df7095d6ef41.jpg",
]

for i, img_path in enumerate(example_images):
    image = cv.imread(img_path)
    ax[i].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    ax[i].axis('off')

plt.show()

In [ ]:
# Set up YOLOv5
!mkdir -p /kaggle/working/tmp
%cd /kaggle/working/tmp
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

In [ ]:
!mkdir -p /kaggle/working/tmp
%cd /kaggle/working/tmp
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install ultralytics

In [ ]:
# Verify setup
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Create the `data.yaml` file for YOLOv5
data_yaml = {
    'path': '/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['Lionfish']
}

# Save the YAML file in the current working directory
with open('/kaggle/working/tmp/yolov8/data.yaml', 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

In [ ]:
# Train YOLOv5
!wandb disabled
!python train.py --img 416 --batch 16 --epochs 50 --data /kaggle/working/tmp/yolov8/data.yaml --weights yolov5n.pt

In [ ]:
# Visualize validation predictions
val_pred_img = "runs/train/exp/val_batch0_pred.jpg"  # Adjust if the folder name differs
if os.path.exists(val_pred_img):
    img = cv.imread(val_pred_img)
    plt.figure(figsize=(15, 15))
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
# Test the model on a sample image
test_image = "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/test/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-0_jpg.rf.06941a03cabb063caa373573523f7fdd.jpg"  # Replace with an actual test image path
!python detect.py --source $test_image --weights runs/train/exp/weights/best.pt --conf 0.25


In [ ]:
# Visualize the detection result
detect_result = "/kaggle/input/lionfish-object-detection-dataset/lionfish-dataset/test/images/Facts_-The-Lionfish---YouTube---Google-Chrome-2021-11-05-15-41-10_mp4-0_jpg.rf.06941a03cabb063caa373573523f7fdd.jpg"  # Adjust if the folder name differs
if os.path.exists(detect_result):
    img = cv.imread(detect_result)
    plt.figure(figsize=(15, 15))
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()